In [1]:
import torchio as tio
from torchio.data.io import sitk_to_nib
import SimpleITK as sitk

from classes.SAMMed3DClassStable import SAMMed3DWrapper, SAMMed3DInferer
from utils.analysisUtils import compute_dice
from utils.modelUtils import load_sammed3d
from utils.promptUtils import get_pos_clicks3D
import utils.analysisUtils as anUt

/home/t722s/anaconda3/envs/universalModels/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/t722s/anaconda3/envs/universalModels/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def get_img_gt_sammed3d(img_path, gt_path):    
    infer_transform = [
        tio.ToCanonical(),
    ]
    transform = tio.Compose(infer_transform)

    sitk_image = sitk.ReadImage(img_path)
    sitk_label = sitk.ReadImage(gt_path)

    if sitk_image.GetOrigin() != sitk_label.GetOrigin():
        sitk_image.SetOrigin(sitk_label.GetOrigin())
    if sitk_image.GetDirection() != sitk_label.GetDirection():
        sitk_image.SetDirection(sitk_label.GetDirection())

    sitk_image_arr, _ = sitk_to_nib(sitk_image)
    sitk_label_arr, _ = sitk_to_nib(sitk_label)

    subject = tio.Subject(
        image = tio.ScalarImage(tensor=sitk_image_arr),
        label = tio.LabelMap(tensor=sitk_label_arr),
    )

    if transform:
        subject = transform(subject)

    return subject.image.data.clone().detach(), subject.label.data.clone().detach()

In [3]:
# Obtain model, image, gt
device = 'cuda'
sammed3d_checkpoint_path = '/home/t722s/Desktop/UniversalModels/TrainedModels/sam_med3d_turbo.pth'

sam_model_tune = load_sammed3d(sammed3d_checkpoint_path, device = device)
wrapper = SAMMed3DWrapper(sam_model_tune, device)
inferer = SAMMed3DInferer(wrapper)

img_path = '/home/t722s/Desktop/Datasets/preprocessed/spleen/AbdomenAtlasJHU_2img/imagesTr/BDMAP_00000001.nii.gz'
gt_path = '/home/t722s/Desktop/Datasets/preprocessed/spleen/AbdomenAtlasJHU_2img/labelsTr/BDMAP_00000001.nii.gz'
img, gt = get_img_gt_sammed3d(img_path, gt_path)


In [4]:
# Experiment: 5 points per volume
seed = 2024
n = 5
pts_prompt = get_pos_clicks3D(gt, n, seed = seed)

#pred = inferer.predict(img, pts_prompt, cheat = True, gt = gt)
segmentation = inferer.predict(img, pts_prompt)

anUt.compute_dice(segmentation, gt.numpy())

0.9264700131652698

In [12]:
# Visualise segmentation slice-wise
slice_idx = 110
anUt.show_seg_row_major(slice_idx, img, gt, segmentation, pts_prompt = pts_prompt)

IndexError: index 110 is out of bounds for dimension 0 with size 1